# Jinja templating system

Terry N. Brown / Brown.TerryN@epa.gov

## https://jinja.palletsprojects.com/en/2.10.x/

## https://github.com/tbnorth/jinja_pres

#### Template
```html
<title>{% block title %}{% endblock %}</title>
<ul>
{% for user in users %}
  <li><a href="{{ user.url }}">{{ user.username }}</a></li>
{% endfor %}
</ul>
```
#### Output
```html
<title>Links to users</title>
<ul>
  <li><a href="http://example.com/user/anne">Anne</a></li>
  <li><a href="http://example.com/user/bob">Bob</a></li>
  <li><a href="http://example.com/user/cal">Cal</a></li>
</ul>
```
(from Jinja docs. website)

## Learn Jinja, learn (part of) Django

Django is a ~full stack web framework for Python, Django's web page
templating system is very similar to Jinja.

## Not just HTML

Markdown uses
```markdown
![Figure caption](./path/to/figure.png){#fig:someLabel}
```

So a template like this:
```markdown
{% for mnth in range(1, 13) %}
![Ten year average particle distribution from each source for
{{D.calendar.month_name[mnth]}}.  Sources are 1-4, top tow, and
6, 7, and "all", bottom row.
]({{'./img/base/mnths/'+D.calendar.month_abbr[mnth]+'.png'}}){#fig:{{D.calendar.month_abbr[mnth]}}}
{% endfor %}
```

Can insert 12 figures like this:
```markdown
![Ten year average particle distribution from each source for
January.  Sources are 1-4, top tow, and
6, 7, and "all", bottom row.
](./img/base/mnths/Jan.png){#fig:Jan}
(+ 11 more figures)
```

markdown ► jinja ► (intermediate markdown) ► pandoc ► PDF    

## Reproducable research / automated reporting

```markdown
Final model results showed {{C.res.PO4ppm * 1000}} ppb of phosphate
across the {{C.sites.n}} sites analyzed.  With an $r^2$ of
{{C.res.r2}} we are...
```

Final model results showed 27 ppb of phosphate
across the 12 sites analyzed.  With an r<sup>2</sup> of
0.91 we are...


## Anything text based

E.g. replace `{{map_name}}` etc. in an SVG graphics file
to produce [hundreds of maps](./site_00026.pdf).

(Yes, [really](https://www.google.com/search?q=bogus%20point&cad=h))

## An example

In [1]:
from jinja2 import Template
import time

# get data from database or other code results
users = [{'name': "Anne", 'id': 11}, {'name': "Bob", 'id': 7}]
asof = time.asctime()

# template is just a string
template = """
User list

(data current {{asof}})

{% for user in users %}
{{user.id}}: {{user.name}}
{% endfor %}
"""

# NOTE: user.id instead of user['id']

# render the template
template = Template(template)
context = dict(users=users, asof=asof)
result = template.render(context)  # also a string
print(result)



User list

(data current Tue Oct  8 08:45:22 2019)


11: Anne

7: Bob



In [2]:
# too many blank lines...
template = """
User list

(data current {{asof}})

{% for user in users -%}
{{user.id}}: {{user.name}}
{% endfor %}
"""
print(Template(template).render(context))


User list

(data current Tue Oct  8 08:45:22 2019)

11: Anne
7: Bob



In [3]:
# Python expressions mostly work
template = """
{% for user in users -%}
{{"%06d" % (user.id+1000)}}: {{user.name}}
{% endfor %}
"""
print(Template(template).render(context))


001011: Anne
001007: Bob



### No computation in the template...

But don't include computation (`user.id+1000`) in the template, it's
considered bad practice, poor separation of concerns.

## More Jinja tools...

In [4]:
# Special loop info. variables, comments
template = """
{% for user in users * 3 %}  {# triple user list -#}
{{loop.index}} {{"%06d" % (user.id+1000)}}: {{user.name}}
{{ loop.cycle('', '', '-'*20+'\n') -}}
{% endfor %}
"""
print(Template(template).render(context))


  1 001011: Anne
  2 001007: Bob
  3 001011: Anne
--------------------
  4 001007: Bob
  5 001011: Anne
  6 001007: Bob
--------------------



In [5]:
# Another special loop info. variable, and an if / endif
template = """
{% for user in users * 3 %}
{{loop.index}} {{"%06d" % (user.id+1000)}}: {{user.name}}
{%- if not loop.last %}{{ loop.cycle('', '', '\n'+'-'*20) }}{% endif -%}
{% endfor %}
"""
print(Template(template).render(context))



1 001011: Anne
2 001007: Bob
3 001011: Anne
--------------------
4 001007: Bob
5 001011: Anne
6 001007: Bob


### `loop.cycle()` is more commonly used in HTML

```html
<table>
  <tr class='odd_row'>...
  <tr class='even_row'>...
```    

In [6]:
# Jinja uses "filters"
context['size'] = 3867529846
template = """
There are {{ users | length }} users
They are using {{ size | filesizeformat }} of storage
"""
print(Template(template).render(context))


There are 2 users
They are using 3.9 GB of storage


## Template loaders
Templates get long and don't belong in .py files

In [7]:
from jinja2 import Environment, FileSystemLoader
import json
env = Environment(loader=FileSystemLoader(['.']))
articles = json.load(open("articles.json"))
# 'articles.json' is a "DB" of articles, 'articles.txt' is the *template*
print(env.get_template("article.txt").render(articles=articles))



EPA Announces Partnership to Reduce Childhood Lead Exposure

10/03/2019

This new MOU provides a framework for a coordinated approach between more than
a dozen critical partners across the federal government, tribes, water
utilities and the public health community. The commitments of the MOU support
the Lead Action Plan, which provides a blueprint for reducing lead exposure and
associated harms by working with a range of stakeholders, including states,
tribes and local communities, along with businesses, property owners and
parents. One existing effort that is further supported by this MOU is EPAâ€™s
3Tsâ€”training, testing and taking actionâ€”for Reducing Lead in Drinking Water
in School and Child Care Facilities.


EPA Recognizes Leaders in Reducing Water Waste

10/07/2019

The City of Charlottesville (Va.) was presented its second Sustained Excellence
Award for rebating WaterSense labeled toilets and providing University of
Virginia student apartments with WaterSense labeled shower

## Blocks and inheritance

**Note:** the `article_general.txt` and `article_gallery.txt` templates handle a single
article, not a list like `article.txt`.

In [8]:
print(env.get_template("article_general.txt").render(article=articles[1]))

EPA Recognizes Leaders in Reducing Water Waste

10/07/2019

The City of Charlottesville (Va.) was presented its second Sustained Excellence
Award for rebating WaterSense labeled toilets and providing University of
Virginia student apartments with WaterSense labeled showerheads and faucet
aerators, which will help save an estimated 60 million gallons of water
annually.
Disclaimer: There is always a disclaimer.

  Home | About | Contact 


In [9]:
print(env.get_template("article_gallery.txt").render(article=articles[1]))

EPA Recognizes Leaders in Reducing Water Waste

10/07/2019

The City of Charlottesville (Va.) was presented its second Sustained Excellence
Award for rebating WaterSense labeled toilets and providing University of
Virginia student apartments with WaterSense labeled showerheads and faucet
aerators, which will help save an estimated 60 million gallons of water
annually.

Gallery
^^^^^^^

[Image: wastebin]
[Image: plastic_bottle]


Disclaimer: There is always a disclaimer.

  Home | About | Contact | Download images



## You can make your own filters

In [10]:
from jinja2 import Environment, DictLoader

def file_size_text(bytes):
    return "way too much"

template = """
There are {{ users | length }} users
They are using {{ size | file_sz_txt }} storage
"""
templates = {
    'main' : template,
    'other': "This is a template"
}

env = Environment(loader=DictLoader(templates))
env.filters['file_sz_txt'] = file_size_text

print(env.get_template('main').render(context))
print('---')
print(env.get_template('other').render(context))



There are 2 users
They are using way too much storage
---
This is a template


## Handling undefined data

In [11]:
template = "You should {{status}} throw the switch."
context['status'] = "definitely NOT"
print(Template(template).render(context))

You should definitely NOT throw the switch.


In [12]:
del context['status']  # status now undefined
print(Template(template).render(context))

You should  throw the switch.


In [13]:
from jinja2 import StrictUndefined
env = Environment(
    loader=DictLoader({'main': template}),
    undefined=StrictUndefined,
)
print(env.get_template('main').render(context))

UndefinedError: 'status' is undefined

In [14]:
# You can also test if something's defined:
template = """
{% if status is defined %}
You should {{status}} throw the switch.
{% else %}
Await further instructions.
{% endif %}
"""
print(Template(template).render(context))



Await further instructions.



## More features

 - Stripping malicious HTML content.
 - Redefine `{{`, `{%` etc. for other applications.
 - Add other extensions.
 - Well documented.